In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
SEED = 42
MODEL_DIR = 'snli_small_sample_default_settings'

In [2]:
import torch 
device = torch.device('cuda:0')

In [3]:
from datasets import load_dataset

# file_dict = {
#   "train" : "../../processed/deu.rst.pcc_train_enriched.rels",
#   "test" : "../../processed/deu.rst.pcc_test_enriched.rels",
#   "valid" : "../../processed/deu.rst.pcc_dev_enriched.rels",
# }

# header = 'doc     unit1_toks      unit2_toks      unit1_txt       unit2_txt       s1_toks s2_toks unit1_sent      unit2_sent      dir     nuc_children    sat_children    genre   u1_discontinuous        u2_discontinuous       u1_issent        u2_issent       u1_length       u2_length       length_ratio    u1_speaker      u2_speaker      same_speaker    u1_func u1_pos  u1_depdir       u2_func u2_pos  u2_depdir       doclen  u1_position      u2_position     percent_distance        distance        lex_overlap_words       lex_overlap_length      unit1_case      unit2_case      label'
# header = header.split()

# dataset = load_dataset(
#   'csv',
#   data_files=file_dict,
#   delimiter=',',
#   column_names=header,
#   skiprows=1
# )

train_dataset = load_dataset('snli', split="train[100:200]")
test_dataset = load_dataset('snli', split="test[:1%]")
valid_dataset = load_dataset('snli', split="validation[:1%]")

train_dataset = train_dataset.filter(lambda example: example["label"]!=-1)
test_dataset = test_dataset.filter(lambda example: example["label"]!=-1)
valid_dataset = valid_dataset.filter(lambda example: example["label"]!=-1)

train_dataset = train_dataset.shuffle(seed=SEED)
test_dataset = test_dataset.shuffle(seed=SEED)
valid_dataset = valid_dataset.shuffle(seed=SEED)
train_dataset.features

Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Loading cached processed dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-37897824cf9337b7.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-b48f2e248a4eaf43.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b/cache-72f53af2d2d7d7db.arrow
Loading ca

{'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradiction'], id=None)}

In [4]:
from transformers import AutoTokenizer, BertTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
class SNLIDataset(torch.utils.data.Dataset):
    """A customized dataset to load the SNLI dataset."""
    def __init__(self, dataset):
        self.text = []
        for premise, hypothesis in zip(dataset['premise'], dataset['hypothesis']):
            self.text.append(tokenizer.encode_plus(premise, hypothesis, padding="max_length", truncation=True, max_length=512))
        self.labels = torch.tensor(dataset['label']).to(device)
        print('read ' + str(len(self.text)) + ' examples')

    def __getitem__(self, idx):
        # return {'input_ids':torch.tensor(self.text[idx]['input_ids']).to(device), 
        #         'token_type_ids':torch.tensor(self.text[idx]['token_type_ids']).to(device), 
        #         'attention_mask':torch.tensor(self.text[idx]['attention_mask']).to(device), 
        #         'label':self.labels[idx]}
        return {'input_ids':self.text[idx]['input_ids'], 
                'token_type_ids':self.text[idx]['token_type_ids'], 
                'attention_mask':self.text[idx]['attention_mask'], 
                'label':self.labels[idx]}

    def __len__(self):
        return len(self.text)


def load_data_snli(batch_size):
    """Download the SNLI dataset and return data iterators and vocabulary."""
    train_data = train_dataset
    valid_data = valid_dataset
    test_data = test_dataset
    train_set = SNLIDataset(train_data)
    valid_set = SNLIDataset(valid_data)
    test_set = SNLIDataset(test_data)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                             shuffle=True)
    valid_iter = torch.utils.data.DataLoader(valid_set, batch_size,
                                            shuffle=False)
    test_iter = torch.utils.data.DataLoader(test_set, batch_size,
                                            shuffle=False)
    
    return train_set, valid_set, test_set
    # return train_iter, valid_iter, test_iter

train_iter, valid_iter, test_iter = load_data_snli(64)

read 99 examples
read 99 examples
read 99 examples


In [18]:
from transformers import AutoModelForSequenceClassification
from transformers.optimization import Adafactor, AdafactorSchedule, AdamW
from torch import optim

model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)
optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
lr_scheduler = AdafactorSchedule(optimizer)
# optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.6, mode='max', patience=7, min_lr=5e-7, verbose=True)#original factor=0.6, min_lr=5e-7

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_vers

In [19]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=MODEL_DIR, 
                                evaluation_strategy="epoch",
                                num_train_epochs=2,
                                save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
                                learning_rate=2e-5,
                                weight_decay=0.01,
                                logging_steps=1)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_iter,
    eval_dataset=valid_iter,
    compute_metrics=compute_metrics,
    optimizers=[optimizer, lr_scheduler]
)

In [26]:
trainer.train() #49, 56, 57

***** Running training *****
  Num examples = 99
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 26
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,1.028000,1.093436,0.363636
2,1.178400,1.090892,0.373737


Trainer is attempting to log a value of "9.374091547442731e-08" of type <class 'torch.Tensor'> for key "train/learning_rate" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "1.4061137676435465e-07" of type <class 'torch.Tensor'> for key "train/learning_rate" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "1.8748183094885462e-07" of type <class 'torch.Tensor'> for key "train/learning_rate" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "2.343522851333546e-07" of type <class 'torch.Tensor'> for key "train/learning_rate" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "2.812227535287093e-07

TrainOutput(global_step=26, training_loss=1.1138092462833111, metrics={'train_runtime': 11.0488, 'train_samples_per_second': 17.92, 'train_steps_per_second': 2.353, 'total_flos': 52096456710144.0, 'train_loss': 1.1138092462833111, 'epoch': 2.0})

In [ ]:
trainer.evaluate(test_iter)

***** Running Evaluation *****
  Num examples = 99
  Batch size = 8


{'eval_loss': 1.0683931112289429,
 'eval_accuracy': 0.42424242424242425,
 'eval_runtime': 1.1348,
 'eval_samples_per_second': 87.239,
 'eval_steps_per_second': 11.456,
 'epoch': 2.0}